# 1. Giriş ve Kavramsal Çerçeve
## Veri Ön İşlemenin Önemi
Veri ön işleme (Data Preprocessing), ham verinin analiz edilebilir ve modellemeye uygun hale getirilmesi sürecidir.
Veri biliminde "Garbage In, Garbage Out" (Çöp içeri, çöp dışarı) prensibi geçerlidir: Veriniz ne kadar kötüyse, modeliniz de o kadar kötü olur.

### Neden Önemli?
- **Veri Kalitesi:** Gerçek dünya verileri genellikle eksik, gürültülü ve tutarsızdır.
- **Model Başarısı:** Makine öğrenmesi algoritmaları temiz ve standart formatta veri bekler.
- **Dönüştürme:** Veriyi algoritmaların anlayacağı sayısal formata çevirmek gerekir.

### Kullanılacak Araçlar
Bu eğitimde aşağıdaki Python kütüphanelerini kullanacağız:
- **Pandas:** Veri manipülasyonu ve analizi.
- **NumPy:** Sayısal hesaplamalar.
- **Matplotlib / Seaborn:** Veri görselleştirme.
- **Scikit-learn:** Ön işleme algoritmaları.


# 2. Kütüphanelerin Yüklenmesi ve Veri Türleri


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

print("Kütüphaneler başarıyla yüklendi.")


## Veri Türleri
Veri setlerinde karşılaşacağımız temel veri türleri:
- **Sayısal (Numerical):** Örn: Yaş, Fiyat (Sürekli veya Kesikli).
- **Kategorik (Categorical):** Örn: Cinsiyet (Nominal), Eğitim Seviyesi (Ordinal).
- **Metin (Text):** İşlenmesi gereken serbest metinler.
- **Tarih-Zaman (Datetime):** Zaman serisi analizleri için.


In [ ]:
# Örnek bir veri seti oluşturalım
data = {
    'Isim': ['Ahmet', 'Ayşe', 'Mehmet', 'Fatma', 'Ali', 'Zeynep', 'Mustafa', 'Elif', 'Burak', 'Ceren'],
    'Yas': [25, 30, np.nan, 22, 28, 35, np.nan, 27, 40, 32],
    'Maas': [5000, 7000, 6000, 4500, np.nan, 8000, 5500, np.nan, 9000, 7500],
    'Sehir': ['Istanbul', 'Ankara', 'Izmir', 'Istanbul', 'Bursa', 'Istanbul', 'Ankara', 'Izmir', 'Bursa', 'Istanbul'],
    'Egitim': ['Lisans', 'Yuksek Lisans', 'Lisans', 'Lise', 'Lisans', 'Doktora', 'Lise', 'Lisans', 'Yuksek Lisans', 'Lisans'],
    'Cinsiyet': ['E', 'K', 'E', 'K', 'E', 'K', 'E', 'K', 'E', 'K'],
    'Tarih': pd.date_range(start='2023-01-01', periods=10, freq='D')
}

df = pd.DataFrame(data)
df.head()


# 3. Veri Setinin Tanınması (Exploratory Data Understanding)
Veriyi tanımak için temel fonksiyonları kullanırız.


In [ ]:
print("Veri Seti Boyutu:", df.shape)
print("\nTemel Bilgiler:")
print(df.info())
print("\nİstatistiksel Özet:")
print(df.describe())
print("\nEksik Değer Kontrolü:")
print(df.isnull().sum())


# 4. Eksik Veriler (Missing Values)
Eksik veriler (NaN) modelin çalışmasını engelleyebilir veya yanıltabilir.
**Türleri:**
- MCAR (Missing Completely at Random): Tamamen rastgele.
- MAR (Missing at Random): Diğer değişkenlerle ilişkili.
- MNAR (Missing Not at Random): Eksiklik verinin kendisiyle ilgili.

**Yöntemler:**
1. Silme (Drop)
2. Doldurma (Imputation: Mean, Median, Mode)


In [ ]:
# Görselleştirme
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Eksik Veri Haritası')
plt.show()

# İmputer ile Doldurma (Ortalama ile)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df[['Yas', 'Maas']] = imputer.fit_transform(df[['Yas', 'Maas']])

print("Eksik veriler doldurulduktan sonra:")
print(df.isnull().sum())
df.head()


# 5. Aykırı Değerler (Outliers)
Veri setindeki genel dağılımdan aşırı sapan değerlerdir.
**Tespit:** Boxplot, Z-Skoru, IQR (Interquartile Range).


In [ ]:
# Aykırı değer ekleyelim
df.loc[9, 'Maas'] = 50000  # Aşırı yüksek bir maaş

# Boxplot ile görselleştirme
plt.figure(figsize=(8, 4))
sns.boxplot(x=df['Maas'])
plt.title('Maaş Değişkeni Boxplot')
plt.show()

# IQR Yöntemi
Q1 = df['Maas'].quantile(0.25)
Q3 = df['Maas'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Maas'] < lower_bound) | (df['Maas'] > upper_bound)]
print(f"Tespit edilen aykırı değerler:\n{outliers}")

# Aykırı değeri baskılama (Capping) veya silme yapılabilir.
# Biz burada üst sınıra eşitleyelim (Capping)
df.loc[df['Maas'] > upper_bound, 'Maas'] = upper_bound
print("\nAykırı değer düzeltildi.")


# 6. Veri Temizleme (Data Cleaning)
Yinelenen kayıtlar, hatalı girişler ve string temizliği.


In [ ]:
# Yinelenen kayıt örneği
duplicate_row = df.iloc[0:1]
df = pd.concat([df, duplicate_row], ignore_index=True)

print(f"Yinelenen kayıt sayısı: {df.duplicated().sum()}")

# Yinelenenleri silme
df.drop_duplicates(inplace=True)
print("Yinelenenler silindi.")

# String temizleme (Büyük/Küçük harf, boşluk)
df['Sehir'] = df['Sehir'].str.lower().str.strip()
print(df['Sehir'].unique())


# 7. Kategorik Verilerin Dönüştürülmesi (Encoding)
Makine öğrenmesi modelleri sadece sayısal veri ile çalışır.
- **Label Encoding:** Ordinal veriler için (Sıralama önemli ise).
- **One-Hot Encoding:** Nominal veriler için (Sıralama önemsiz ise).


In [ ]:
# Label Encoding (Egitim: Sıralı veri)
le = LabelEncoder()
# Sıralamayı biz belirleyelim: Lise < Lisans < Yuksek Lisans < Doktora
egitim_map = {'Lise': 1, 'Lisans': 2, 'Yuksek Lisans': 3, 'Doktora': 4}
df['Egitim_Encoded'] = df['Egitim'].map(egitim_map)

# One-Hot Encoding (Sehir: Sırasız)
df_encoded = pd.get_dummies(df, columns=['Sehir', 'Cinsiyet'], drop_first=True)
# drop_first=True ile Dummy Variable Trap'ten kaçınıyoruz

print(df_encoded.head())


# 8. Sayısal Verilerin Ölçeklenmesi (Scaling)
Farklı birimlerdeki (Yaş: 0-100, Maaş: 2000-50000) verileri aynı ölçeğe getirmek.
- **Min-Max Scaling:** 0 ile 1 arasına sıkıştırır.
- **Standard Scaler:** Ortalaması 0, standart sapması 1 olacak şekilde dönüştürür.


In [ ]:
scaler = StandardScaler()
df_encoded[['Yas', 'Maas']] = scaler.fit_transform(df_encoded[['Yas', 'Maas']])

print("Ölçeklenmiş Veri:")
print(df_encoded[['Yas', 'Maas']].head())


# 9. Veri Dönüşümleri (Transformations)
Çarpık dağılımları normal dağılıma yaklaştırmak için Log veya Karekök dönüşümü yapılır.


In [ ]:
# Çarpık veri örneği
df_encoded['Carpik_Deger'] = np.random.exponential(scale=2, size=len(df))

# Log Dönüşümü
df_encoded['Log_Carpik'] = np.log1p(df_encoded['Carpik_Deger'])

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.histplot(df_encoded['Carpik_Deger'], kde=True)
plt.title('Orijinal (Çarpık)')

plt.subplot(1, 2, 2)
sns.histplot(df_encoded['Log_Carpik'], kde=True)
plt.title('Log Dönüşümü Sonrası')
plt.show()


# 10. Özellik Seçimi ve Özellik Mühendisliği (Feature Engineering)
Yeni özellikler türetmek ve en önemlilerini seçmek.
- **Yeni Özellik:** `Tecrube_Puani = Yas * Egitim_Encoded` (Örnek)
- **Korelasyon:** Gereksiz veya birbiriyle çok ilişkili özellikleri elemek.


In [ ]:
# Yeni Özellik Türetme
df_encoded['Tecrube_Puani'] = df_encoded['Yas'] * df_encoded['Egitim_Encoded']

# Korelasyon Matrisi
plt.figure(figsize=(10, 8))
# Sadece sayısal sütunları seçelim
numeric_cols = df_encoded.select_dtypes(include=[np.number]).columns
sns.heatmap(df_encoded[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Korelasyon Matrisi')
plt.show()

print("Veri Ön İşleme Eğitimi Tamamlandı! 🎉")
